In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
from src.inference import get_feature_store

/home/saideshith1905/venv3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]

2025-03-05 02:54:57,763 INFO: Initializing external client
2025-03-05 02:54:57,764 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 02:54:58,668 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215689
Fetching data from 2025-02-04 07:54:57.762419+00:00 to 2025-03-05 06:54:57.762419+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (6.30s) 


In [5]:
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)

,pickup_hour,pickup_location_id,rides
0,2025-02-04 08:00:00+00:00,2,0
1,2025-02-04 09:00:00+00:00,2,0
2,2025-02-04 10:00:00+00:00,2,0
3,2025-02-04 11:00:00+00:00,2,0
4,2025-02-04 12:00:00+00:00,2,0
...,...,...,...
175112,2025-03-05 02:00:00+00:00,263,5
175113,2025-03-05 03:00:00+00:00,263,3
175114,2025-03-05 04:00:00+00:00,263,4
175115,2025-03-05 05:00:00+00:00,263,19


In [6]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 175117 entries, 0 to 176622
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype                  
---  ------              --------------   -----                  
 0   pickup_hour         175117 non-null  datetime64[us, Etc/UTC]
 1   pickup_location_id  175117 non-null  int32                  
 2   rides               175117 non-null  int32                  
dtypes: datetime64[us, Etc/UTC](1), int32(2)
memory usage: 4.0 MB


In [7]:
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

In [8]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 175117 entries, 0 to 176622
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   pickup_hour         175117 non-null  datetime64[us]
 1   pickup_location_id  175117 non-null  int32         
 2   rides               175117 non-null  int32         
dtypes: datetime64[us](1), int32(2)
memory usage: 4.0 MB


In [9]:
from src.data_utils import transform_ts_data_info_features
features = transform_ts_data_info_features(ts_data, window_size=24*28, step_size=23)

Skipping location_id 176: Not enough data to create even one window.


In [10]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,1,3,0,1,0,1,0,3,0,0,...,0,4,0,2,1,0,3,4,188,2025-02-04 08:00:00
1,1,1,0,0,1,1,2,1,2,0,...,0,0,0,0,0,0,0,1,196,2025-02-04 11:00:00
2,32,136,124,24,123,145,229,57,130,85,...,57,25,106,41,132,125,21,124,263,2025-02-04 17:00:00
3,2,0,1,3,0,0,24,0,0,0,...,2,0,0,1,3,0,2,1,255,2025-02-04 14:00:00
4,0,0,0,1,0,0,0,1,1,0,...,8,0,0,1,0,1,0,0,112,2025-02-04 22:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,0,0,1,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,200,2025-02-05 02:00:00
247,0,1,0,0,1,0,0,0,0,0,...,1,0,0,0,0,1,0,0,122,2025-02-04 14:00:00
248,67,70,69,81,1,75,122,165,22,93,...,60,124,121,74,124,4,70,55,231,2025-02-04 20:00:00
249,0,0,0,0,0,1,0,0,0,1,...,2,0,0,2,2,0,2,0,205,2025-02-04 17:00:00


In [11]:
from src.inference import load_batch_of_features_from_store
current_date = pd.Timestamp.now(tz='Etc/UTC')
features = load_batch_of_features_from_store(current_date)

2025-03-05 02:55:15,772 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-05 02:55:15,776 INFO: Initializing external client
2025-03-05 02:55:15,777 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 02:55:16,373 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215689
Fetching data from 2025-02-04 07:55:15.772144+00:00 to 2025-03-05 06:55:15.772144+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (5.35s) 
Skipping location_id 176: Not enough data to create even one window.


In [12]:
current_date

Timestamp('2025-03-05 07:55:15.772144+0000', tz='Etc/UTC')

In [13]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,2025-03-04 08:00:00
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3,2025-03-04 08:00:00
2,11,7,3,2,3,2,1,7,5,3,...,4,4,3,0,0,1,5,16,4,2025-03-04 08:00:00
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,6,2025-03-04 08:00:00
4,1,2,3,2,3,1,2,4,1,0,...,0,1,0,0,0,0,5,7,7,2025-03-04 08:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,2,259,2025-03-04 08:00:00
247,2,0,1,1,1,0,1,1,0,0,...,0,1,0,0,0,1,1,2,260,2025-03-04 08:00:00
248,5,16,21,34,25,31,37,28,33,48,...,1,0,0,1,1,1,1,9,261,2025-03-04 08:00:00
249,177,156,102,81,88,81,79,97,95,105,...,2,2,1,4,4,16,87,155,262,2025-03-04 08:00:00


In [14]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-03-05 02:55:28,256 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-05 02:55:28,260 INFO: Initializing external client
2025-03-05 02:55:28,261 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 02:55:28,852 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215689

In [15]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

In [16]:
predictions

,pickup_location_id,predicted_demand
0,2,0.0
1,3,0.0
2,4,9.0
3,6,0.0
4,7,4.0
...,...,...
246,259,0.0
247,260,1.0
248,261,8.0
249,262,169.0


In [17]:
predictions.sort_values("predicted_demand", ascending=False).head(10)["pickup_location_id"].values

array([236, 186, 237, 239, 162, 142, 140, 170, 262, 238], dtype=int32)